<h1 align="center"><b> Galapagos Seals Data Collection </b></h1>

In this notebook, we consolidate our dataset of labeled images of the two species of Galapagos Sea Lions. All our images will be obtained from The Global Biodiversity Information Facility. The procedure is as follows:

1.  Visit [https://www.gbif.org](https://www.gbif.org)
2.  Create an account.
3.  Go to `Get Data > Species`. And type the scientific name of the desired animal. 🔍
4.  Go to `Gallery > Download > Darwin Core Archive`.
5.  You will receive an email with a download link. We will use this link in this notebook.

**Downloading Images of Arctocephalus Galapagoensis**


We'll begin by downloading the images for Arctocephalus Galapagoensis (Galapagos Fur Seal). This is the first species we'll focus on in our dataset consolidation. 📸

In [ ]:
# Step 1: Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Define your download directory in Drive
import os
import zipfile
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm # For a nice progress bar

# Define the main directory in your Drive for all GBIF downloads
gbif_root_dir = '/content/drive/My Drive/gbif_raw_downloads2'
os.makedirs(gbif_root_dir, exist_ok=True)

# Define a specific directory for the Galapagos Fur Seal dataset
dataset_name = 'Arctocephalus_galapagoensis_DwC-A'
dataset_drive_path = os.path.join(gbif_root_dir, dataset_name)
os.makedirs(dataset_drive_path, exist_ok=True)
print(f"Dataset will be processed in: {dataset_drive_path}")

# --- IMPORTANT: Paste your GBIF Download Link here (Generate one following the manual download procedure specified in GBIF webpage) ---
GBIF_DOWNLOAD_URL = '----'


dwca_filename = os.path.join(dataset_drive_path, 'Arctocephalus_galapagoensis_dataset.zip')

print(f"Downloading DwC-A to: {dwca_filename}")
# Download the zip file directly to your Drive
# The -O flag saves the file with the specified name
!wget -O "{dwca_filename}" "{GBIF_DOWNLOAD_URL}"

print("\nDwC-A download complete. Starting extraction...")

# Unzip the downloaded DwC-A directly in Drive
try:
    with zipfile.ZipFile(dwca_filename, 'r') as zip_ref:
        zip_ref.extractall(dataset_drive_path)
    print("DwC-A extracted successfully.")
except zipfile.BadZipFile:
    print("Error: The downloaded file is not a valid zip file. Please check the GBIF_DOWNLOAD_URL.")
    exit()
except Exception as e:
    print(f"An error occurred during unzipping: {e}")
    exit()

# Path to the multimedia.txt file within the extracted archive
multimedia_file = os.path.join(dataset_drive_path, 'multimedia.txt')

if not os.path.exists(multimedia_file):
    print(f"Error: {multimedia_file} not found. Ensure the DwC-A contained it.")
    print("This usually means no media records were found for your search criteria on GBIF.")
else:
    # Read the multimedia.txt file into a Pandas DataFrame
    multimedia_df = pd.read_csv(multimedia_file, sep='\t', low_memory=False)

    print(f"Loaded {len(multimedia_df)} records from multimedia.txt")
    print("\nColumns available in multimedia.txt:")
    print(multimedia_df.columns.tolist()) # <--- IMPORTANT: Print available columns

    # Determine which column to use for organizing images
    # We prefer 'taxonKey', then 'scientificName', otherwise a generic folder
    image_label_column = None
    if 'taxonKey' in multimedia_df.columns:
        image_label_column = 'taxonKey'
    elif 'scientificName' in multimedia_df.columns: # As a fallback for organization
        image_label_column = 'scientificName'
    else:
        print("\nWarning: Neither 'taxonKey' nor 'scientificName' found in multimedia.txt.")
        print("Images will be saved to a single folder without species-specific subfolders.")

    # Filter for 'StillImage' (photos) and relevant columns
    # 'identifier' is the image URL
    # 'license' is crucial for legal use
    required_cols = ['identifier', 'license', 'type']
    if image_label_column:
        required_cols.append(image_label_column)

    # Ensure all required columns exist before selecting
    missing_cols = [col for col in required_cols if col not in multimedia_df.columns]
    if missing_cols:
        print(f"Error: Missing required columns in multimedia.txt: {missing_cols}")
        print("Please check your GBIF download settings or the file content.")
        exit()

    image_data = multimedia_df[
        (multimedia_df['type'] == 'StillImage') &
        (multimedia_df['identifier'].notna())
    ][required_cols].dropna(subset=['identifier'])

    # Filter to only keep images for the specific taxonKey if 'taxonKey' is present
    ARCTOCEPHALUS_GALAPAGOENSIS_TAXON_KEY = 2433473
    if image_label_column == 'taxonKey':
        image_data = image_data[image_data['taxonKey'] == ARCTOCEPHALUS_GALAPAGOENSIS_TAXON_KEY]
        print(f"Found {len(image_data)} 'StillImage' URLs for TaxonKey {ARCTOCEPHALUS_GALAPAGOENSIS_TAXON_KEY}.")
    elif image_label_column == 'scientificName':
        # If using scientificName, you might want to filter by the name as well
        image_data = image_data[image_data['scientificName'] == 'Arctocephalus galapagoensis']
        print(f"Found {len(image_data)} 'StillImage' URLs for scientificName 'Arctocephalus galapagoensis'.")
    else:
        print(f"Found {len(image_data)} 'StillImage' URLs (no specific taxonKey filter applied).")


    if len(image_data) == 0:
        print("No images found matching your criteria in the multimedia.txt file.")
    else:
        # Create a directory to store the actual image files
        image_output_dir = os.path.join(dataset_drive_path, 'images')
        os.makedirs(image_output_dir, exist_ok=True)

        # Define the final image saving directory
        if image_label_column:
            # Create a subfolder based on taxonKey or scientificName
            # Make sure to convert label to string for path creation
            save_dir_name = str(ARCTOCEPHALUS_GALAPAGOENSIS_TAXON_KEY) if image_label_column == 'taxonKey' else 'Arctocephalus_galapagoensis'
            final_image_save_dir = os.path.join(image_output_dir, save_dir_name)
        else:
            final_image_save_dir = os.path.join(image_output_dir, 'unlabeled_images') # Fallback

        os.makedirs(final_image_save_dir, exist_ok=True)
        print(f"Images will be saved to: {final_image_save_dir}")

        # --- Function to download a single image ---
        def download_image(row):
            image_url = row['identifier']
            image_name = os.path.basename(image_url).split('?')[0]
            if '.' not in image_name: # Add a default extension if none present
                image_name += '.jpg'

            filepath = os.path.join(final_image_save_dir, image_name)

            if os.path.exists(filepath):
                return f"Skipped: {filepath} already exists."

            try:
                response = requests.get(image_url, stream=True, timeout=15)
                response.raise_for_status()

                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                return f"Downloaded: {filepath}"
            except requests.exceptions.Timeout:
                return f"Failed to download {image_url}: Timeout."
            except requests.exceptions.ConnectionError:
                return f"Failed to download {image_url}: Connection error."
            except requests.exceptions.RequestException as e:
                status_code = response.status_code if 'response' in locals() else 'N/A'
                return f"Failed to download {image_url}: HTTP Error {status_code} - {e}"
            except Exception as e:
                return f"An unexpected error occurred for {image_url}: {e}"

        # --- Concurrent downloading ---
        max_workers = 10

        print(f"\nStarting concurrent image download with {max_workers} workers...")
        results = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(download_image, row) for index, row in image_data.iterrows()]

            for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading Images"):
                results.append(future.result())

        print("\nImage download process complete. Sample results:")
        for res in results[:min(5, len(results))]:
            print(res)
        if len(results) > 5:
            print("...")

        downloaded_count = sum(1 for r in results if r.startswith("Downloaded:"))
        skipped_count = sum(1 for r in results if r.startswith("Skipped:"))
        failed_count = len(results) - downloaded_count - skipped_count

        print(f"\nSummary:")
        print(f"  Successfully downloaded: {downloaded_count}")
        print(f"  Skipped (already exists): {skipped_count}")
        print(f"  Failed: {failed_count}")
        print(f"  Total URLs processed: {len(results)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset will be processed in: /content/drive/My Drive/gbif_raw_downloads2/Arctocephalus_galapagoensis_DwC-A
--2025-05-20 21:21:53--  https://api.gbif.org/v1/occurrence/download/request/0010447-250515123054153.zip
Resolving api.gbif.org (api.gbif.org)... 130.225.43.2
Connecting to api.gbif.org (api.gbif.org)|130.225.43.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://occurrence-download.gbif.org/occurrence/download/request/0010447-250515123054153.zip [following]
--2025-05-20 21:21:53--  https://occurrence-download.gbif.org/occurrence/download/request/0010447-250515123054153.zip
Resolving occurrence-download.gbif.org (occurrence-download.gbif.org)... 130.225.43.36
Connecting to occurrence-download.gbif.org (occurrence-download.gbif.org)|130.225.43.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length


Image download process complete. Sample results:
Downloaded: /content/drive/My Drive/gbif_raw_downloads2/Arctocephalus_galapagoensis_DwC-A/images/unlabeled_images/original.jpg
Skipped: /content/drive/My Drive/gbif_raw_downloads2/Arctocephalus_galapagoensis_DwC-A/images/unlabeled_images/original.jpg already exists.
Downloaded: /content/drive/My Drive/gbif_raw_downloads2/Arctocephalus_galapagoensis_DwC-A/images/unlabeled_images/original.jpg
Downloaded: /content/drive/My Drive/gbif_raw_downloads2/Arctocephalus_galapagoensis_DwC-A/images/unlabeled_images/original.jpg
Downloaded: /content/drive/My Drive/gbif_raw_downloads2/Arctocephalus_galapagoensis_DwC-A/images/unlabeled_images/original.jpg
...

Summary:
  Successfully downloaded: 72
  Skipped (already exists): 520
  Failed: 0
  Total URLs processed: 592


 **Downloading Images of Zalophus Wallebaeki**


We proceed by downloading the images for Zalophus Wallebaeki (Galapagos Seal Lion). 📸

In [ ]:
# Step 1: Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Define your download directory in Drive
import os
import zipfile
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm # For a nice progress bar

# Define the main directory in your Drive for all GBIF downloads
gbif_root_dir = '/content/drive/My Drive/gbif_raw_downloads'
os.makedirs(gbif_root_dir, exist_ok=True)

# Define a specific directory for the Zalophus wollebaeki dataset
dataset_name = 'Zalophus_wollebaeki_DwC-A'
dataset_drive_path = os.path.join(gbif_root_dir, dataset_name)
os.makedirs(dataset_drive_path, exist_ok=True)
print(f"Dataset will be processed in: {dataset_drive_path}")

# --- IMPORTANT: PASTE YOUR NEW GBIF DOWNLOAD LINK FOR ZALOPHUS WOLLEBAEKI HERE ---
GBIF_DOWNLOAD_URL = '----'
# Example: GBIF_DOWNLOAD_URL = 'https://api.gbif.org/v1/occurrence/download/request/0012345-250502131345914.zip'


dwca_filename = os.path.join(dataset_drive_path, 'Zalophus_wollebaeki_dataset.zip')

print(f"Downloading DwC-A to: {dwca_filename}")
# Download the zip file directly to your Drive
!wget -O "{dwca_filename}" "{GBIF_DOWNLOAD_URL}"

print("\nDwC-A download complete. Starting extraction...")

# Unzip the downloaded DwC-A directly in Drive
try:
    with zipfile.ZipFile(dwca_filename, 'r') as zip_ref:
        zip_ref.extractall(dataset_drive_path)
    print("DwC-A extracted successfully.")
except zipfile.BadZipFile:
    print("Error: The downloaded file is not a valid zip file. Please check the GBIF_DOWNLOAD_URL.")
    exit()
except Exception as e:
    print(f"An error occurred during unzipping: {e}")
    exit()

# Path to the multimedia.txt file within the extracted archive
multimedia_file = os.path.join(dataset_drive_path, 'multimedia.txt')

if not os.path.exists(multimedia_file):
    print(f"Error: {multimedia_file} not found. Ensure the DwC-A contained it.")
    print("This usually means no media records were found for your search criteria on GBIF.")
else:
    # Read the multimedia.txt file into a Pandas DataFrame
    multimedia_df = pd.read_csv(multimedia_file, sep='\t', low_memory=False)

    print(f"Loaded {len(multimedia_df)} records from multimedia.txt")
    print("\nColumns available in multimedia.txt:")
    print(multimedia_df.columns.tolist()) # <--- IMPORTANT: Print available columns

    # Determine which column to use for organizing images
    image_label_column = None
    if 'taxonKey' in multimedia_df.columns:
        image_label_column = 'taxonKey'
    elif 'scientificName' in multimedia_df.columns:
        image_label_column = 'scientificName'
    else:
        print("\nWarning: Neither 'taxonKey' nor 'scientificName' found in multimedia.txt.")
        print("Images will be saved to a single folder without species-specific subfolders.")

    # Filter for 'StillImage' (photos) and relevant columns
    required_cols = ['identifier', 'license', 'type']
    if image_label_column:
        required_cols.append(image_label_column)

    # Ensure all required columns exist before selecting
    missing_cols = [col for col in required_cols if col not in multimedia_df.columns]
    if missing_cols:
        print(f"Error: Missing required columns in multimedia.txt: {missing_cols}")
        print("Please check your GBIF download settings or the file content.")
        exit()

    image_data = multimedia_df[
        (multimedia_df['type'] == 'StillImage') &
        (multimedia_df['identifier'].notna())
    ][required_cols].dropna(subset=['identifier'])

    # Filter to only keep images for the specific taxonKey if 'taxonKey' is present
    # This is the taxonKey for Zalophus wollebaeki
    ZALOPHUS_WOLLEBAEKI_TAXON_KEY = 5218765
    if image_label_column == 'taxonKey':
        image_data = image_data[image_data['taxonKey'] == ZALOPHUS_WOLLEBAEKI_TAXON_KEY]
        print(f"Found {len(image_data)} 'StillImage' URLs for TaxonKey {ZALOPHUS_WOLLEBAEKI_TAXON_KEY}.")
    elif image_label_column == 'scientificName':
        image_data = image_data[image_data['scientificName'] == 'Zalophus wollebaeki']
        print(f"Found {len(image_data)} 'StillImage' URLs for scientificName 'Zalophus wollebaeki'.")
    else:
        print(f"Found {len(image_data)} 'StillImage' URLs (no specific taxonKey/scientificName filter applied).")


    if len(image_data) == 0:
        print("No images found matching your criteria in the multimedia.txt file.")
    else:
        # Create a directory to store the actual image files
        image_output_dir = os.path.join(dataset_drive_path, 'images')
        os.makedirs(image_output_dir, exist_ok=True)

        # Define the final image saving directory
        if image_label_column:
            # Create a subfolder based on taxonKey or scientificName
            save_dir_name = str(ZALOPHUS_WOLLEBAEKI_TAXON_KEY) if image_label_column == 'taxonKey' else 'Zalophus_wollebaeki'
            final_image_save_dir = os.path.join(image_output_dir, save_dir_name)
        else:
            final_image_save_dir = os.path.join(image_output_dir, 'unlabeled_images') # Fallback

        os.makedirs(final_image_save_dir, exist_ok=True)
        print(f"Images will be saved to: {final_image_save_dir}")

        # --- Function to download a single image ---
        def download_image(row):
            image_url = row['identifier']
            image_name = os.path.basename(image_url).split('?')[0]
            if '.' not in image_name:
                image_name += '.jpg'

            filepath = os.path.join(final_image_save_dir, image_name)

            if os.path.exists(filepath):
                return f"Skipped: {filepath} already exists."

            try:
                response = requests.get(image_url, stream=True, timeout=15)
                response.raise_for_status()

                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                return f"Downloaded: {filepath}"
            except requests.exceptions.Timeout:
                return f"Failed to download {image_url}: Timeout."
            except requests.exceptions.ConnectionError:
                return f"Failed to download {image_url}: Connection error."
            except requests.exceptions.RequestException as e:
                status_code = response.status_code if 'response' in locals() else 'N/A'
                return f"Failed to download {image_url}: HTTP Error {status_code} - {e}"
            except Exception as e:
                return f"An unexpected error occurred for {image_url}: {e}"

        # --- Concurrent downloading ---
        max_workers = 10

        print(f"\nStarting concurrent image download with {max_workers} workers...")
        results = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(download_image, row) for index, row in image_data.iterrows()]

            for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading Images"):
                results.append(future.result())

        print("\nImage download process complete. Sample results:")
        for res in results[:min(5, len(results))]:
            print(res)
        if len(results) > 5:
            print("...")

        downloaded_count = sum(1 for r in results if r.startswith("Downloaded:"))
        skipped_count = sum(1 for r in results if r.startswith("Skipped:"))
        failed_count = len(results) - downloaded_count - skipped_count

        print(f"\nSummary:")
        print(f"  Successfully downloaded: {downloaded_count}")
        print(f"  Skipped (already exists): {skipped_count}")
        print(f"  Failed: {failed_count}")
        print(f"  Total URLs processed: {len(results)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset will be processed in: /content/drive/My Drive/gbif_raw_downloads/Zalophus_wollebaeki_DwC-A
--2025-05-20 21:48:21--  https://api.gbif.org/v1/occurrence/download/request/0010490-250515123054153.zip
Resolving api.gbif.org (api.gbif.org)... 130.225.43.2
Connecting to api.gbif.org (api.gbif.org)|130.225.43.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://occurrence-download.gbif.org/occurrence/download/request/0010490-250515123054153.zip [following]
--2025-05-20 21:48:22--  https://occurrence-download.gbif.org/occurrence/download/request/0010490-250515123054153.zip
Resolving occurrence-download.gbif.org (occurrence-download.gbif.org)... 130.225.43.36
Connecting to occurrence-download.gbif.org (occurrence-download.gbif.org)|130.225.43.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspeci


Image download process complete. Sample results:
Skipped: /content/drive/My Drive/gbif_raw_downloads/Zalophus_wollebaeki_DwC-A/images/unlabeled_images/original.jpeg already exists.
Skipped: /content/drive/My Drive/gbif_raw_downloads/Zalophus_wollebaeki_DwC-A/images/unlabeled_images/original.jpg already exists.
Skipped: /content/drive/My Drive/gbif_raw_downloads/Zalophus_wollebaeki_DwC-A/images/unlabeled_images/original.jpg already exists.
Skipped: /content/drive/My Drive/gbif_raw_downloads/Zalophus_wollebaeki_DwC-A/images/unlabeled_images/original.jpeg already exists.
Skipped: /content/drive/My Drive/gbif_raw_downloads/Zalophus_wollebaeki_DwC-A/images/unlabeled_images/original.jpeg already exists.
...

Summary:
  Successfully downloaded: 326
  Skipped (already exists): 6467
  Failed: 0
  Total URLs processed: 6793


---

###  Dataset Organization

Our dataset will be organized into three distinct sets for model training, validation, and testing (80-10-10):

#### --- TRAIN Set ---
* *Arctocephalus galapagoensis*: 41 images
* *Zalophus wollebaeki*: 240 images

#### --- VALIDATION Set ---
* *Arctocephalus galapagoensis*: 5 images
* *Zalophus wollebaeki*: 30 images

#### --- TEST Set ---
* *Arctocephalus galapagoensis*: 6 images
* *Zalophus wollebaeki*: 30 images

---

In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split # A convenient way to split data

# Ensure Google Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

print("Starting dataset organization...")

# --- 1. Define Paths and Class Mappings (UPDATED SOURCE PATHS) ---

# Base directory where your downloaded images are
base_source_dir = '/content/drive/My Drive/gbif_raw_downloads/'

# Mapping of taxonKey (used for organization) to human-readable class name
# The 'source_path' now points to the 'unlabeled_images' folder within each species' download
species_info = {
    '2433473': { # TaxonKey for Arctocephalus galapagoensis
        'name': 'Arctocephalus_galapagoensis',
        'source_path': os.path.join(base_source_dir, 'Arctocephalus_galapagoensis_DwC-A', 'images', 'unlabeled_images')
    },
    '5218765': { # TaxonKey for Zalophus wollebaeki
        'name': 'Zalophus_wollebaeki',
        'source_path': os.path.join(base_source_dir, 'Zalophus_wollebaeki_DwC-A', 'images', 'unlabeled_images')
    }
}

# New organized dataset base directory in your Drive
organized_dataset_base_dir = '/content/drive/My Drive/my_galapagos_seals_dataset'
os.makedirs(organized_dataset_base_dir, exist_ok=True)


# --- 2. Define Split Ratios ---
train_split_ratio = 0.8
val_split_ratio = 0.1
test_split_ratio = 0.1


# --- 3. Create the New Dataset Structure ---
subdirs = ['train', 'validation', 'test']

for subdir in subdirs:
    for species_key, info in species_info.items():
        class_name = info['name']
        target_path = os.path.join(organized_dataset_base_dir, subdir, class_name)
        os.makedirs(target_path, exist_ok=True)
        print(f"Created directory: {target_path}")

print("\nNew dataset directory structure created.")

# --- 4. Populate the Dataset ---

print("\nStarting to copy and split images...")

for species_key, info in species_info.items():
    species_name = info['name']
    source_folder = info['source_path']
    print(f"\nProcessing {species_name} from: {source_folder}")

    # Get all image file paths for the current species
    image_files = [f for f in os.listdir(source_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]
    random.shuffle(image_files) # Shuffle to ensure random split

    total_images = len(image_files)
    print(f"Found {total_images} images for {species_name}.")

    if total_images == 0:
        print(f"No images found in {source_folder}. Skipping {species_name}.")
        continue

    # Perform the split using sklearn's train_test_split
    # First split: train vs. (validation + test)
    train_files, val_test_files = train_test_split(
        image_files,
        train_size=train_split_ratio,
        random_state=42, # For reproducibility
        shuffle=True
    )

    # Second split: validation vs. test from the remaining files
    if len(val_test_files) > 0:
        # Calculate test_size relative to the remaining val_test_files
        test_size_relative = test_split_ratio / (val_split_ratio + test_split_ratio)
        val_files, test_files = train_test_split(
            val_test_files,
            test_size=test_size_relative,
            random_state=42,
            shuffle=True
        )
    else:
        val_files = []
        test_files = []

    print(f"  Train: {len(train_files)} images")
    print(f"  Validation: {len(val_files)} images")
    print(f"  Test: {len(test_files)} images")


    # Copy files to their respective directories
    for filename in train_files:
        src = os.path.join(source_folder, filename)
        dst = os.path.join(organized_dataset_base_dir, 'train', species_name, filename)
        shutil.copyfile(src, dst)

    for filename in val_files:
        src = os.path.join(source_folder, filename)
        dst = os.path.join(organized_dataset_base_dir, 'validation', species_name, filename)
        shutil.copyfile(src, dst)

    for filename in test_files:
        src = os.path.join(source_folder, filename)
        dst = os.path.join(organized_dataset_base_dir, 'test', species_name, filename)
        shutil.copyfile(src, dst)

print("\nDataset organization complete!")

# --- 5. Verify Counts (Optional but Recommended) ---
print("\nVerifying final counts in the new dataset structure:")
for subdir in subdirs:
    print(f"\n--- {subdir.upper()} Set ---")
    for species_key, info in species_info.items():
        species_name = info['name']
        target_path = os.path.join(organized_dataset_base_dir, subdir, species_name)
        count = len(os.listdir(target_path))
        print(f"  {species_name}: {count} images")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting dataset organization...
Created directory: /content/drive/My Drive/my_galapagos_seals_dataset/train/Arctocephalus_galapagoensis
Created directory: /content/drive/My Drive/my_galapagos_seals_dataset/train/Zalophus_wollebaeki
Created directory: /content/drive/My Drive/my_galapagos_seals_dataset/validation/Arctocephalus_galapagoensis
Created directory: /content/drive/My Drive/my_galapagos_seals_dataset/validation/Zalophus_wollebaeki
Created directory: /content/drive/My Drive/my_galapagos_seals_dataset/test/Arctocephalus_galapagoensis
Created directory: /content/drive/My Drive/my_galapagos_seals_dataset/test/Zalophus_wollebaeki

New dataset directory structure created.

Starting to copy and split images...

Processing Arctocephalus_galapagoensis from: /content/drive/My Drive/gbif_raw_downloads/Arctocephalus_galapagoensis_DwC-A/images/unlabeled_images
Fou